Limpeza dos Dados

In [5]:
import pandas as pd

# Carregar os dados
data = pd.read_csv('spotify_dataset.csv')

# Para colunas categóricas, preencha com a moda
for column in data.select_dtypes(include=['object']).columns:
    data[column].fillna(data[column].mode()[0], inplace=True)

# Para colunas numéricas, você pode calcular o IQR
numeric_data = data.select_dtypes(exclude=['object'])

Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)
IQR = Q3 - Q1

# Verifique outliers apenas para colunas numéricas
outliers = ((numeric_data < (Q1 - 1.5 * IQR)) | (numeric_data > (Q3 + 1.5 * IQR)))

# Crie um DataFrame apenas com os outliers
outliers_data = data[outliers.any(axis=1)]

# Remover outliers do DataFrame original
data = data[~outliers.any(axis=1)]

# Agora, `outliers_data` contém os outliers retirados
print(outliers_data)



                                  Nome da Faixa         Cantor ou Compositor  \
1                           Reborn in Blasphemy                    Dismember   
2                           Ultrasonic Meltdown  Curse of the Golden Vampire   
4                        The Masquerade is Over              George Shearing   
7                                      All Good                   De La Soul   
8                               Rats!Rats!Rats!                     Deftones   
..                                          ...                          ...   
698                         Comfort Zone; Pt. 1               Steven Halpern   
699  I Don't Stand a Ghost of a Chance With You                  Sonny Stitt   
700                     Let the Computer Decide                 Paul Gilbert   
701                          Brother Harold Dee               Porter Wagoner   
705                                        Read                      Product   

                                  Gener

In [7]:
data.describe()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.00000,403.000000,403.000000,403.0,403.000000
mean,0.385971,0.564052,0.593888,0.033499,4.875931,0.168267,-9.383814,0.69727,0.062610,113.681469,4.0,0.573616
std,0.353441,0.175003,0.263202,0.084140,3.472635,0.105219,4.127859,0.46001,0.037838,26.475275,0.0,0.269599
min,0.000001,0.114000,0.024200,0.000000,0.000000,0.026800,-21.229000,0.00000,0.023900,53.564000,4.0,0.040000
25%,0.028650,0.442500,0.394500,0.000002,2.000000,0.095400,-11.827000,0.00000,0.035300,94.755500,4.0,0.345500
50%,0.289000,0.598000,0.626000,0.000131,5.000000,0.124000,-9.122000,1.00000,0.046800,110.017000,4.0,0.618000
75%,0.724500,0.692000,0.817500,0.009605,7.500000,0.231500,-6.080500,1.00000,0.077550,130.863500,4.0,0.799500
max,0.996000,0.913000,0.998000,0.422000,11.000000,0.492000,-1.577000,1.00000,0.179000,195.959000,4.0,0.980000


## Codificação de Categorias

In [10]:
# Primeiro, vamos identificar as colunas categóricas
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()

# Remover as colunas "Nome da Faixa" e "Cantor ou Compositor" da lista
columns_to_exclude = ['Nome da Faixa', 'Cantor ou Compositor']
for col in columns_to_exclude:
    if col in categorical_columns:
        categorical_columns.remove(col)

# Verificar se as colunas foram removidas da lista
print("Colunas categóricas para codificação:", categorical_columns)

# Agora, vamos aplicar a codificação one-hot apenas às colunas selecionadas
data_encoded = pd.get_dummies(data, columns=categorical_columns)

print(data_encoded.head())  # exibir as primeiras linhas para ver a transformação
print("Total de colunas após a codificação:", len(data_encoded.columns))




Colunas categóricas para codificação: ['Genero', 'Quadrante']
                     Nome da Faixa      Cantor ou Compositor  acousticness  \
0  Bulldog Down in Sunny Tennessee             Charlie Poole        0.9600   
3                       Last Night             Little Walter        0.3580   
5                Little Saint Nick            The Beach Boys        0.1080   
6                   I Call It Love           Concrete Blonde        0.1960   
9                           Exodus  Bob Marley & the Wailers        0.0301   

   danceability  energy  instrumentalness  key  liveness  loudness  mode  ...  \
0         0.649   0.390          0.000003    5    0.0960   -12.237     1  ...   
3         0.458   0.443          0.000000    6    0.2320   -10.422     0  ...   
5         0.602   0.553          0.000000   11    0.0512    -9.336     1  ...   
6         0.432   0.934          0.000029    0    0.4920    -5.651     1  ...   
9         0.868   0.744          0.005170    7    0.1720    -8.5

1. Expansão dos Valores Compostos:

In [11]:
# Criar um novo dataframe expandindo a coluna "Genero"
expanded_data = data.assign(Genero=data['Genero'].str.split(';')).explode('Genero')

# Remove espaços em branco adicionais (caso existam)
expanded_data['Genero'] = expanded_data['Genero'].str.strip()


2. Aplicar Codificação One-hot:

In [12]:
# Selecionar as colunas categóricas que você deseja codificar
cols_to_encode = ['Genero', 'Quadrante']

# Aplicar codificação one-hot
encoded_data = pd.get_dummies(expanded_data, columns=cols_to_encode, drop_first=False)

print(encoded_data.head())
print("Total de colunas após a codificação:", encoded_data.shape[1])


                     Nome da Faixa Cantor ou Compositor  acousticness  \
0  Bulldog Down in Sunny Tennessee        Charlie Poole         0.960   
0  Bulldog Down in Sunny Tennessee        Charlie Poole         0.960   
3                       Last Night        Little Walter         0.358   
5                Little Saint Nick       The Beach Boys         0.108   
5                Little Saint Nick       The Beach Boys         0.108   

   danceability  energy  instrumentalness  key  liveness  loudness  mode  ...  \
0         0.649   0.390          0.000003    5    0.0960   -12.237     1  ...   
0         0.649   0.390          0.000003    5    0.0960   -12.237     1  ...   
3         0.458   0.443          0.000000    6    0.2320   -10.422     0  ...   
5         0.602   0.553          0.000000   11    0.0512    -9.336     1  ...   
5         0.602   0.553          0.000000   11    0.0512    -9.336     1  ...   

   Genero_R&B  Genero_Rap  Genero_Reggae  Genero_Religious  \
0       Fals

In [14]:
# Agrupando e somando os valores
df_agrupado = encoded_data.groupby(['Nome da Faixa', 'Cantor ou Compositor']).sum().reset_index()

# Convertendo valores maiores que 1 de volta para 1 (caso haja alguma soma que resultou em um valor > 1)
for col in df_agrupado.columns:
    if 'Genero_' in col or 'Quadrante_' in col:
        df_agrupado[col] = df_agrupado[col].apply(lambda x: 1 if x > 1 else x)


### Normalização

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Criando o objeto scaler
scaler = MinMaxScaler()

# Lista de colunas numéricas para normalizar
# (Adicione ou remova colunas da lista conforme necessário)
colunas_numericas = [
    'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
    'loudness', 'mode', 'speechiness', 'tempo', 'valence'
]

# Aplicando a normalização
df_agrupado[colunas_numericas] = scaler.fit_transform(df_agrupado[colunas_numericas])
